In [1]:
import sqlite3
import pandas as pd
import numpy as np
from gurobipy import *
import datetime
import time

from Wanderworld import *

def generateNewRoute(model):
    route_tuples = []

    for (key,value) in model.LEGS.iteritems():
        if (value.x!=0) and key[0] != key[1]:
            #print key, cost[key], str(key[2])[-2:]
            route_tuples.append(key+(model.COST[key],))

    #print ""

    #print route_tuples
        
    route_tuples = sorted(route_tuples, key=lambda tup: tup[2]) #sorts on date
    route_tuples.append(model.MODEL.objVal) #adds price to list
    route_tuples = tuple(route_tuples) #transforms list to tuple
    
    return route_tuples

    #for i in route_tuples:
        #print i



In [2]:
conn = sqlite3.connect("flights_final_4_13.db")
c = conn.cursor()
query = c.execute('''SELECT * From flights_min_city ''')
cols = [column[0] for column in query.description]
results = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [6]:
results[results['origin_city_name']=='Berlin']

,price,origin_id,destination_id,outbound_date,inbound_date,currency,origin_city_name,origin_country_name,destination_city_name,destination_country_name,direct,carrier_name,carrier_id,skyscanner_code,date_collected
4200,35.0,TXL,DUS,2017-05-05T00:00:00,2017-05-05T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 13:02:15.332427
4201,35.0,TXL,DUS,2017-06-12T00:00:00,2017-06-12T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081
4202,35.0,TXL,DUS,2017-07-07T00:00:00,2017-07-07T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 09:30:09.017505
4203,35.0,TXL,DUS,2017-08-18T00:00:00,2017-08-18T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081
4656,36.0,TXL,DUS,2017-06-10T00:00:00,2017-06-10T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081
4657,36.0,TXL,DUS,2017-06-14T00:00:00,2017-06-14T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081
4658,36.0,TXL,DUS,2017-07-19T00:00:00,2017-07-19T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 09:30:09.017505
4950,37.0,TXL,DUS,2017-05-04T00:00:00,2017-05-04T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081
4951,37.0,TXL,DUS,2017-05-09T00:00:00,2017-05-09T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081
4952,37.0,TXL,DUS,2017-05-17T00:00:00,2017-05-17T00:00:00,USD,Berlin,Germany,Dusseldorf,Germany,1,Air Berlin,834,TXL,2017-04-13 00:12:06.629081


In [17]:
if __name__ == "__main__":
	conn = sqlite3.connect("flight_sample.db")
	c = conn.cursor()
	query = c.execute('''SELECT * From flights ''')
	cols = [column[0] for column in query.description]
	results = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

	# results.to_csv('flights.csv')

	#df = pd.read_sql_query("SELECT * FROM table_name", conn)
	#dat = sqlite3.connect("flight.db")
	#query = dat.execute("SELECT * From flight")
	#cols = [column[0] for column in query.description]
	#results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)


	user_start = '2017-05-01T00:00:00'
	user_end = '2017-05-31T00:00:00'
	origins = results['origin_id'].unique()
	city_list = origins
	num_days = 15
	min_days = 2
	origin = 'ATL'
	#print city_list


	Example = WanderWorld(results)
	Example.inputTimeFrame(user_start, user_end, city_list, num_days, min_days, origin)
	Example.createInputDicts()
	Example.buildAndSolveModel()

	routes = []

	for model in Example.modelList:
		routes.append(generateNewRoute(model))

	#for model in Example.modelList:
    	#in main loop:
    	#routes.append(generateNewRoute(model))
    	#print routes
		#print routes

    	#sort and slice at very end:
	sorted_routes = sorted(routes, key=lambda tup: tup[-1]) #sorts on total price
	final_results = sorted_routes[0:4] #or however many we want to return

	# print 'length:',len(final_results)
	# print final_results

	#import csv
	#with open('final_result.csv', 'wb') as myfile:
	#	wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
	#	wr.writerow(final_results)
    
	outputDict = {}
	outputDict['TripID'] = []
	outputDict['Seq'] = []
	outputDict['Origin'] = []
	outputDict['Dest'] = []
	outputDict['Price'] = []
	outputDict['Date1'] = []
	outputDict['Date2'] = []
	outputDict['Total Price'] = [] 
    
	for tripID, route in enumerate(final_results):
		#print route
		totalPrice = route[-1]
		for seq, flight in enumerate(route): 
			if(flight==totalPrice):
				break
			outputDict['TripID'].append(tripID)
			outputDict['Seq'].append(seq)
			outputDict['Origin'].append(flight[0])
			outputDict['Dest'].append(flight[1])
			outputDict['Price'].append(flight[4])
			departDate_String, arriDate_String = extractRealDate(flight)
			outputDict['Date1'].append(departDate_String)
			outputDict['Date2'].append(arriDate_String)
			outputDict['Total_Price'].append(totalPrice) 
            
	print outputDict
	outputDF = pd.DataFrame.from_dict(outputDict)
	print outputDF
    
	outputDF.to_csv('niceOutput.csv') 
    




2017-05-01
2017-05-02
2017-05-03
2017-05-04
2017-05-05
2017-05-06
2017-05-07
2017-05-08
2017-05-09
2017-05-10
2017-05-11
2017-05-12
2017-05-13
2017-05-14
2017-05-15
2017-05-16
Optimize a model with 54 rows, 125 columns and 414 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [3e+01, 3e+03]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+01]
Presolve removed 10 rows and 27 columns
Presolve time: 0.01s
Presolved: 44 rows, 98 columns, 332 nonzeros
Variable types: 0 continuous, 98 integer (98 binary)
Found heuristic solution: objective 4051.0000000

Root relaxation: objective 2.092061e+03, 59 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2092.06061    0   26 4051.00000 2092.06061  48.4%     -    0s
H    0     0                    2106.0000000 2092.06061  0.66%     -    0s
     0     0     cutoff    0

KeyError: 'Total_Price'

In [10]:
def extractRealDate(flight):
    departDate = flight[2]
    arriDate = flight[3]
    departDate_String = departDate.strftime("%Y-%m-%d")# %H:%M:%S")
    arriDate_String = arriDate.strftime("%Y-%m-%d")
    
    return departDate_String, arriDate_String
    